In [63]:
import numpy as np
import pandas as pd
import math
import time

# Start timer
start = time.perf_counter()

# 导入数据
#copy excel all before run code运行前粘贴表9
df=pd.read_clipboard("\t")
print(df)

     陈列类      细类代码      细类名称           商品代码  \
0     食品  32016289  饼干/糕点/膨化  2011768410107   
1     食品  32016289  饼干/糕点/膨化  2011768210103   
2     食品  32016289  饼干/糕点/膨化  2015343610104   
3     食品  32016289  饼干/糕点/膨化  2012176910104   
4     食品  32016289  饼干/糕点/膨化  2015246010100   
..   ...       ...       ...            ...   
915  儿童袜  29013356       儿童袜  2013876311109   
916  儿童袜  29013356       儿童袜  2013876410109   
917  儿童袜  29013356       儿童袜  2013876411106   
918  儿童袜  29013356       儿童袜  2013876510106   
919  儿童袜  29013356       儿童袜  2013876511103   

                                         商品名称  系列名称      版权描述  价格带   季节  \
0    美国自采: ORION TURTLE CHIP CHOCO CHURROS （L   未分配       非版权  未分配  未分配   
1    美国自采: ORION TURTLE CHIP CORN SOUP （L） 16   未分配       非版权  未分配  未分配   
2       美国自采: ORION TURTLE CHIPS MALA （M） 80G   未分配       非版权  未分配  未分配   
3     美国自采: ORION TURTLE CHIPS, SEAWEED, 160G   未分配       非版权  未分配  未分配   
4    美国自采: HELLO KITTY WAFER COOKIES GREEN TE   未分配       非版

In [64]:
df['商品代码'] = [str(x) for x in df['商品代码'] ]

In [65]:
# 每个店需要根据下单计划进行手改
store_code = 'US23'
date = '230731'

# 提前下单日期-手改
advance_day = 2

# 目标下单金额-手改每次
#stock_amount =55000

# 目标下单pieces-手改每次
stock_piece = 6875

In [66]:
# sub-factor 次级参数

# 可售天数
turnover= 58+advance_day 
big_sell_well_day = 60+advance_day # 畅销品最大补货周转天数
patch_Upper_and_lower = 500 #补货数量上下限

balance_ratio = 1.05 # 细类平衡率 
standard_number_of_day_up=28+advance_day #标准订货日期上限 30
# area_ratio = 0.8 # 区域日均销比率
sell_well_down=0.3 #畅销补货底线 0.3
sell_well_down_all_area = 0.5 #综合、区域畅销补货底线 0.45
standard_number_of_day_down = 38+advance_day # 低于最低日期的能补的最高日起  43
new_well_down = 0.30  # 新品补货底线 0.3
mini_balance = 1.3  # 细类平衡率偏差 1.4
max_sale_day = 78+advance_day # 调整品类的最大的周转天数  78
slender_deviate = 100 # 细类最大偏差  100
add_to_down = 0.35 #追加补货底线 0.35
wheel_number = 5 # 最后降序每循环多少次刷新一次

In [67]:
# top_article=150 #补畅销品数量

# west arriving date 7; east arriving date 14
west_arriving_date=7+advance_day
east_arriving_data=14+advance_day

In [68]:
sale_number_1 =0
df1 = df

In [69]:
df1.shape[0]

920

In [70]:
# 判断有无其实是数字的字符串
# 调整格式

list_1 = ['仓库\n可用库存', '门店\n库存','系统\n订货数量', '调整后\n订货数量'
        , ' 订货包数', '总销量', '综合\n日均销', '本店\n日均销',  '区域\n日均销'
        , '可售天数', '单品标准PCS', '细类定标\nPCS', '细类偏差\nPCS'
        ,'细类\n平衡率']


# 带#的单品
str_to_int = []
if_have_jing = False
for i in range(df1.shape[0]):
    if df1['本地\n组合价'][i]=='#':
        df1['本地\n组合价'][i] = 0
        df1['整箱规格'][i]='0*0*0'
        df1['订货\n类型'][i]='禁配'
        if_have_jing = True
        
if if_have_jing:
    for i in range(df1.shape[0]):   
        if type(df1['本地\n组合价'][i]) == str and '.'in df1['本地\n组合价'][i]:
            str_to_int.append(float(df1['本地\n组合价'][i]))
        else:
            str_to_int.append(df1['本地\n组合价'][i])
    df1['本地\n组合价'] = str_to_int
        

# 门店库存转数字

for i in range(len(list_1)):
    str_to_int = []
    for j in range(df1.shape[0]): 
        if pd.isnull(df1[list_1[i]][j]):
            str_to_int.append(0)
        elif type(df1[list_1[i]][j]) == int:
            str_to_int.append(df1[list_1[i]][j])
        elif type(df1[list_1[i]][j]) == np.int64:
            str_to_int.append(df1[list_1[i]][j])
        elif type(df1[list_1[i]][j]) == np.float64:
            str_to_int.append(df1[list_1[i]][j])
        elif type(df1[list_1[i]][j]) == str and '%'in df1[list_1[i]][j]:
            if ','in df1[list_1[i]][j]:
                str_to_int.append(float(str(df1[list_1[i]][j]).replace(',','').replace('%',''))*0.01)
            else:
                str_to_int.append(float(str(df1[list_1[i]][j]).replace('%',''))*0.01)
        elif type(df1[list_1[i]][j]) == str and ','in df1[list_1[i]][j]:
            str_to_int.append(float(str(df1[list_1[i]][j]).replace(',','')))
        elif type(df1[list_1[i]][j]) == str and '.'in df1[list_1[i]][j]:
            str_to_int.append(float(df1[list_1[i]][j]))
        elif type(df1[list_1[i]][j]) == str and '.'not in df1[list_1[i]][j] and ','not in df1[list_1[i]][j]:
            str_to_int.append(float(df1[list_1[i]][j]))        
            
    df1[list_1[i]] = str_to_int
  

In [71]:
#规格转换
#读取原箱规格1*1*1
size = []
for i in range(df1.shape[0]):
    size.append(int(df1['整箱规格'][i].split("*", 3)[1]))

df1['规格'] = size

In [72]:
# 预处理表9
# 剔除禁配
sale_number = []  # 下单包数
for i in range(df1.shape[0]):
    if df1['订货\n原因'][i] == '禁配':
        sale_number.append(0)
    else:
        sale_number.append(df1[' 订货包数'][i])
        
# 剔除仓库库存=<0 or 订货包数=<0

sale_number = []  # 下单包数

for i in range(df1.shape[0]):
    if df1['仓库\n可用库存'][i] <= 0:
        sale_number.append(0)
    elif df1['仓库\n可用库存'][i] > 0 and df1['仓库\n可用库存'][i]<df1[' 订货包数'][i]*df1['规格'][i]:
        sale_number.append(df1['仓库\n可用库存'][i]/df1['规格'][i])
    else:
        sale_number.append(df1[' 订货包数'][i])
        
df1[' 订货包数'] =sale_number

In [73]:
# # 剔除出系统下单下多的，把订货数量的销售天数控制在turnoverday以内

# sale_number = []  # 下单包数
# sale_day_1 = []   # 可售天数

# for i in range(df1.shape[0]):
    
#     if df1['仓库\n可用库存'][i] > 0:
#         if df1[' 订货包数'][i] >0 and df1['可售天数'][i]>turnover and df1['订货\n类型'][i]!='新品':
#             sale_number_1 = df1[' 订货包数'][i]
#             while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] > turnover:
#                 sale_number_1 = sale_number_1 - 1
#             sale_number.append(sale_number_1)
#             sale_day_1.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i])
#         else:
#             sale_number.append(df1[' 订货包数'][i])
#             sale_day_1.append(df1['可售天数'][i])
#     else:
#         sale_number.append(df1[' 订货包数'][i])
#         sale_day_1.append(df1['可售天数'][i])

# df1['订货包数'] =sale_number
# df1['可售天数'] = sale_day_1
# df1['调整后\n订货数量']= df1[' 订货包数']*df1['规格']

In [74]:
# 处理car系统自动补货数量
# 剔除出系统下单下多的，剔除car系统补货后可售天数过高的
sale_number = []
sale_day_1 = []

for i in range(df1.shape[0]):
    if df1['仓库\n可用库存'][i] > 0:
        if df1[' 订货包数'][i] > 0 and df1['订货\n类型'][i]!='新品':
            if df1['陈列类'][i] == '居家生活-公仔' and df1['可售天数'][i]>40:
                sale_number_1 = df1[' 订货包数'][i]
                while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] > 40:
                    sale_number_1 = sale_number_1 - 1
                sale_number.append(sale_number_1)
                sale_day_1.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i])
            elif df1['陈列类'][i] == '食品'and df1['可售天数'][i]>70:
                if df1['细类名称'][i] == '饮料/饮品' and df1['可售天数'][i]>45:
                    sale_number_1 = df1[' 订货包数'][i]
                    while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] > 45:
                        sale_number_1 = sale_number_1 - 1
                    sale_number.append(sale_number_1)
                    sale_day_1.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i])
                else:
                    sale_number_1 = df1[' 订货包数'][i]
                    while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] > 70:
                        sale_number_1 = sale_number_1 - 1
                    sale_number.append(sale_number_1)
                    sale_day_1.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i])
            elif df1['陈列类'][i] == '盲盒系列' and df1['可售天数'][i]>80:
                sale_number_1 = df1[' 订货包数'][i]
                while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] > 80:
                    sale_number_1 = sale_number_1 - 1
                sale_number.append(sale_number_1)
                sale_day_1.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i])
            elif df1['可售天数'][i] > 30:
                sale_number_1 = df1[' 订货包数'][i]
                while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] > 30:
                    sale_number_1 = sale_number_1 - 1
                sale_number.append(sale_number_1)
                sale_day_1.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i])
            else:
                sale_number.append(df1[' 订货包数'][i])
                sale_day_1.append(df1['可售天数'][i])
        else:
            sale_number.append(df1[' 订货包数'][i])
            sale_day_1.append(df1['可售天数'][i])
    else:
        sale_number.append(df1[' 订货包数'][i])
        sale_day_1.append(df1['可售天数'][i])

df1[' 订货包数'] =sale_number
df1['可售天数'] = sale_day_1
df1['调整后\n订货数量']= df1[' 订货包数']*df1['规格']

In [75]:
# 补货总数
sum(df1[' 订货包数']*df1['规格'])

236

In [76]:
# 自主补货
# 1、补三丽鸥产品

sale_number =[]  #调整订货包数
sale_day_1 = []   # 可售天数
    
for i in range(df1.shape[0]):
    if df1['订货\n类型'][i]!='新品' and df1['订货\n原因'][i] != '禁配':
        if df1['版权描述'][i] == '三丽鸥-名创版权':
            sale_number_1 = df1[' 订货包数'][i]
            if sale_number_1 > 0:
                sale_number.append(df1[' 订货包数'][i])
                sale_day_1.append(df1['可售天数'][i]) 
            else:
                if df1['仓库\n可用库存'][i] > 0 and (df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i]<=standard_number_of_day_up and df1['门店\n库存'][i]<df1['规格'][i]/2:
                    if df1['陈列类'][i] == '居家生活-公仔' and df1['可售天数'][i]<=40:
                        while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] <=40 and df1[' 订货包数']*df1['规格'] < df1['仓库\n可用库存']:
                            sale_number_1 = sale_number_1 + 1
                        sale_number.append(sale_number_1)
                        sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i]))
                        df1['订货\n类型'][i] = 0
                    elif df1['陈列类'][i] == '食品' and df1['可售天数'][i]<=70:
                        if df1['细类名称'][i] == '饮料/饮品' and df1['可售天数'][i]<=45:
                            while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] <=45 and df1[' 订货包数']*df1['规格'] < df1['仓库\n可用库存']:
                                sale_number_1 = sale_number_1 + 1
                            sale_number.append(sale_number_1)
                            sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i]))
                            df1['订货\n类型'][i] = 0
                        else:
                            while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] <=70 and df1[' 订货包数']*df1['规格'] < df1['仓库\n可用库存']:
                                sale_number_1 = sale_number_1 + 1
                            sale_number.append(sale_number_1)
                            sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i]))
                            df1['订货\n类型'][i] = 0
                    elif df1['陈列类'][i] == '盲盒系列' and df1['可售天数'][i]<=80:
                        while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] <=80 and df1[' 订货包数']*df1['规格'] < df1['仓库\n可用库存']:
                            sale_number_1 = sale_number_1 + 1
                        sale_number.append(sale_number_1)
                        sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i]))
                        df1['订货\n类型'][i] = 0
                    elif df1['可售天数'][i] <= 30:
                        while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] <=30 and df1[' 订货包数']*df1['规格'] < df1['仓库\n可用库存']:
                            sale_number_1 = sale_number_1 + 1
                        sale_number.append(sale_number_1)
                        sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i]))
                        df1['订货\n类型'][i] = 0
                    else:
                        sale_number.append(df1[' 订货包数'][i])
                        sale_day_1.append(df1['可售天数'][i])
                else:
                    sale_number.append(df1[' 订货包数'][i])
                    sale_day_1.append(df1['可售天数'][i])
        else:
            sale_number.append(df1[' 订货包数'][i])
            sale_day_1.append(df1['可售天数'][i])
    else:
        sale_number.append(df1[' 订货包数'][i])
        sale_day_1.append(df1['可售天数'][i])

df1[' 订货包数']=sale_number
df1['可售天数'] = sale_day_1
df1['调整后\n订货数量']= df1[' 订货包数']*df1['规格']

C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\3236451777.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  if df1['仓库\n可用库存'][i] > 0 and (df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i]<=standard_number_of_day_up and df1['门店\n库存'][i]<df1['规格'][i]/2:
C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\3236451777.py:45: RuntimeWarning: divide by zero encountered in double_scalars
  sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i]))
C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\3236451777.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['订货\n类型'][i] = 0


In [77]:
sale_number =[]  #调整订货包数
sale_day_1 = []   # 可售天数
    
for i in range(df1.shape[0]):
    if df1['订货\n类型'][i]!='新品' and df1['订货\n原因'][i] != '禁配':
        if df1['仓库\n可用库存'][i] > 0 and (df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i]<=standard_number_of_day_up and df1['门店\n库存'][i]<df1['规格'][i]/2:
            if df1['陈列类'][i] == '居家生活-公仔' and df1['可售天数'][i]<=40:
                while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] <=40 and df1[' 订货包数']*df1['规格'] < df1['仓库\n可用库存']:
                    sale_number_1 = sale_number_1 + 1
                sale_number.append(sale_number_1)
                sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i]))
                df1['订货\n类型'][i] = 0
            elif df1['陈列类'][i] == '食品' and df1['可售天数'][i]<=70:
                if df1['细类名称'][i] == '饮料/饮品' and df1['可售天数'][i]<=45:
                    while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] <=45 and df1[' 订货包数']*df1['规格'] < df1['仓库\n可用库存']:
                        sale_number_1 = sale_number_1 + 1
                    sale_number.append(sale_number_1)
                    sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i]))
                    df1['订货\n类型'][i] = 0
                else:
                    while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] <=70 and df1[' 订货包数']*df1['规格'] < df1['仓库\n可用库存']:
                        sale_number_1 = sale_number_1 + 1
                    sale_number.append(sale_number_1)
                    sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i]))
                    df1['订货\n类型'][i] = 0
            elif df1['陈列类'][i] == '盲盒系列' and df1['可售天数'][i]<=80:
                while sale_number_1 > 0 and (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i] <=80 and df1[' 订货包数']*df1['规格'] < df1['仓库\n可用库存']:
                    sale_number_1 = sale_number_1 + 1
                sale_number.append(sale_number_1)
                sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i]))
                df1['订货\n类型'][i] = 0
            else:
                sale_number.append(df1[' 订货包数'][i])
                sale_day_1.append(df1['可售天数'][i]) 
        else:
            sale_number.append(df1[' 订货包数'][i])
            sale_day_1.append(df1['可售天数'][i])
    else:
        sale_number.append(df1[' 订货包数'][i])
        sale_day_1.append(df1['可售天数'][i])

df1[' 订货包数']=sale_number
df1['可售天数'] = sale_day_1
df1['调整后\n订货数量']= df1[' 订货包数']*df1['规格']

C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\3660233520.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['订货\n类型'][i] = 0
C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\3660233520.py:24: RuntimeWarning: divide by zero encountered in double_scalars
  sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i]))
C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\3660233520.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  if df1['仓库\n可用库存'][i] > 0 and (df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i]<=standard_number_of_day_up and df1['门店\n库存'][i]<df1['规格'][i]/2:
C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\3660233520.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  sale_day_1.a

In [78]:
# 2. 补本店畅销品

sale_number =[]#调整订货包数
sale_day_1 = []   # 可售天数
sale_day_type = []  #销售类型

for i in range(df1.shape[0]):
    if df1['仓库\n可用库存'][i] > 0 and df1['订货\n类型'][i]!='新品' and df1['订货\n原因'][i] != '禁配':    
        # if i<top_article:补top商品
        if df1['本店\n日均销'][i] > sell_well_down:
            sale_number_1 = df1[' 订货包数'][i]
            if (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['本店\n日均销'][i]<standard_number_of_day_down:
                while (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['本店\n日均销'][i]<standard_number_of_day_down and df1['仓库\n可用库存'][i] > sale_number_1+1:
                    sale_number_1=sale_number_1+1
    
                if (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['本店\n日均销'][i]<=standard_number_of_day_up:
                    if sale_number_1 >0:
                        sale_number.append(sale_number_1)
                        sale_day_1.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['本店\n日均销'][i])
                        sale_day_type.append(0)
                    else:
                        sale_number.append(0)
                        sale_day_1.append((df1['门店\n库存'][i]+0*df1['规格'][i])/df1['本店\n日均销'][i])
                        sale_day_type.append(0)
                
                elif  (df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['本店\n日均销'][i]>standard_number_of_day_up:
                    if sale_number_1 >0:
                        sale_number.append(sale_number_1-1)
                        sale_day_1.append((df1['门店\n库存'][i]+(sale_number_1-1)*df1['规格'][i])/df1['本店\n日均销'][i])
                        sale_day_type.append(0)
                    
                    else:
                        sale_number.append(0)
                        sale_day_1.append((df1['门店\n库存'][i]+0*df1['规格'][i])/df1['本店\n日均销'][i])
                        sale_day_type.append(0)
            else:
                sale_number.append(sale_number_1)
                sale_day_1.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['本店\n日均销'][i])
                sale_day_type.append(0)
        else:
            sale_number.append(df1[' 订货包数'][i])
            sale_day_1.append(df1['可售天数'][i])
            sale_day_type.append(0)

    else:
        sale_number.append(df1[' 订货包数'][i])
        sale_day_1.append(df1['可售天数'][i])
        sale_day_type.append(0)
    
df1['订货包数']=sale_number
df1['可售天数'] = sale_day_1
df1['订货\n类型'] = sale_day_type
df1['调整后\n订货数量']= df1[' 订货包数']*df1['规格']

In [79]:
sum(df1[' 订货包数']*df1['规格'])

236

In [80]:
# 3. 补综合畅销

sale_number =[]#调整订货包数
sale_day_1 = []   # 可售天数
    
for i in range(df1.shape[0]):
    if df1['订货\n类型'][i]!='新品' and df1['订货\n原因'][i] != '禁配':
        # if i<top_article:  #补top的
        if df1['综合\n日均销'][i] > sell_well_down_all_area:
            sale_number_1 = df1[' 订货包数'][i]
            if sale_number_1 > 0:
                sale_number.append(df1[' 订货包数'][i])
                sale_day_1.append(df1['可售天数'][i]) 
            else:
                if df1['仓库\n可用库存'][i] > 0 and (df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i]<=standard_number_of_day_up and df1['门店\n库存'][i]<df1['规格'][i]/2 and df1['细类\n平衡率'][i] < balance_ratio:
                    sale_number.append(1)
                    sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i])) 
                    df1['订货\n类型'][i] = 0
                else:
                    sale_number.append(df1[' 订货包数'][i])
                    sale_day_1.append(df1['可售天数'][i]) 
        else:
            sale_number.append(df1[' 订货包数'][i])
            sale_day_1.append(df1['可售天数'][i])
    else:
        sale_number.append(df1[' 订货包数'][i])
        sale_day_1.append(df1['可售天数'][i])

df1[' 订货包数']=sale_number
df1['可售天数'] = sale_day_1
df1['调整后\n订货数量']= df1[' 订货包数']*df1['规格']

C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\2202897740.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['订货\n类型'][i] = 0
C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\2202897740.py:17: RuntimeWarning: divide by zero encountered in double_scalars
  sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i]))


In [81]:
sum(df1[' 订货包数']*df1['规格'])

591

In [82]:
# 4. 补区域畅销

sale_number =[]#调整订货包数
sale_day_1 = []   # 可售天数
    
for i in range(df1.shape[0]):
    if df1['订货\n类型'][i]!='新品' and df1['订货\n原因'][i] != '禁配':
        # if i<top_article:  #补top的
        if df1['区域\n日均销'][i] > sell_well_down_all_area:
            sale_number_1 = df1[' 订货包数'][i]
            if sale_number_1 > 0:
                sale_number.append(df1[' 订货包数'][i])
                sale_day_1.append(df1['可售天数'][i]) 
            else:
                if df1['仓库\n可用库存'][i] > 0 and (df1['门店\n库存'][i]+1*df1['规格'][i])/df1['区域\n日均销'][i]<=standard_number_of_day_up and df1['门店\n库存'][i]<df1['规格'][i]/2 and df1['细类\n平衡率'][i] < balance_ratio:
                    sale_number.append(1)
                    sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['区域\n日均销'][i])) 
                    df1['订货\n类型'][i] = 2
                else:
                    sale_number.append(df1[' 订货包数'][i])
                    sale_day_1.append(df1['可售天数'][i]) 
        else:
            sale_number.append(df1[' 订货包数'][i])
            sale_day_1.append(df1['可售天数'][i])
    else:
        sale_number.append(df1[' 订货包数'][i])
        sale_day_1.append(df1['可售天数'][i])

df1[' 订货包数']=sale_number
df1['可售天数'] = sale_day_1
df1['调整后\n订货数量']= df1[' 订货包数']*df1['规格']

C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\2259298872.py:17: RuntimeWarning: divide by zero encountered in double_scalars
  sale_day_1.append(min((df1['门店\n库存'][i]+1*df1['规格'][i])/df1['本店\n日均销'][i],(df1['门店\n库存'][i]+1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['区域\n日均销'][i]))
C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\2259298872.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['订货\n类型'][i] = 2


In [83]:
sum(df1[' 订货包数']*df1['规格'])

611

In [84]:
# 5. 补新品

sale_number =[]#调整订货包数
sale_day_1 = []   # 可售天数

for i in range(df1.shape[0]):
    if df1['仓库\n可用库存'][i] > 0 and df1['订货\n类型'][i]=='新品' and df1['订货\n原因'][i] != '禁配':
        sale_number_1 = df1[' 订货包数'][i]
        if df1['系统\n订货数量'][i] > 0:
            sale_number_1= df1['系统\n订货数量'][i]/df1['规格'][i]
            sale_number.append(sale_number_1)
            sale_day_1.append(0)
            df1['订货\n类型'][i] = -1
        else:
            if (df1['综合\n日均销'][i]>new_well_down or df1['区域\n日均销'][i]>new_well_down) and df1['细类\n平衡率'][i]<1:
                while sale_number_1*df1['规格'][i] < df1[ '单品标准PCS'][i] and df1['仓库\n可用库存'][i] >= (sale_number_1+1)*df1['规格'][i] :
                    sale_number_1=sale_number_1+1
                sale_number.append(sale_number_1)
                sale_day_1.append(min((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['本店\n日均销'][i],(df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i],(df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['区域\n日均销'][i]))
                df1['订货\n类型'][i]=-1
            else:
                sale_number.append(0)
                sale_day_1.append(0)
    else:
        sale_number.append(df1[' 订货包数'][i])
        sale_day_1.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i])  
        
        #if df1['补货类型'][i] == 0:
            #sale_day_1.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/max(df1['本店\n日均销'][i],df1['综合\n日均销'][i],df1['区域\n日均销'][i]))         
        #elif df1['补货类型'][i] == 1:
            #sale_day_1.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/max(df1['本店\n日均销'][i],df1['综合\n日均销'][i],df1['区域\n日均销'][i]))       
        #else:
            #sale_day_1.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/max(df1['本店\n日均销'][i],df1['综合\n日均销'][i],df1['区域\n日均销'][i]))      

df1[' 订货包数']=sale_number
df1['可售天数'] = sale_day_1
df1['调整后\n订货数量']= df1[' 订货包数']*df1['规格']

C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\227124675.py:26: RuntimeWarning: divide by zero encountered in double_scalars
  sale_day_1.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i])


In [85]:
sum(df1[' 订货包数']*df1['规格'])

611

In [86]:
# 6. 先补有销售但超天数的
for i in range(df1.shape[0]):
    if 0 < ((df1['门店\n库存'][i]+df1[' 订货包数'][i]*df1['规格'][i])/df1['本店\n日均销'][i]) < standard_number_of_day_down-advance_day and df1['本店\n日均销'][i] > sell_well_down and (df1['门店\n库存'][i]+(df1[' 订货包数'][i]+1)*df1['规格'][i])/df1['本店\n日均销'][i] < standard_number_of_day_down-advance_day and df1['仓库\n可用库存'][i] >= (df1[' 订货包数'][i] + 1)*df1['规格'][i] and df1['订货\n类型'][i]!='新品' and df1['订货\n原因'][i] != '禁配':
        df1[' 订货包数'][i] = df1[' 订货包数'][i] + 1
        df1['订货\n类型'][i] = 3
        
old_sum = sum(df1[' 订货包数']*df1['规格'])
print(old_sum)

923


C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\2634855962.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[' 订货包数'][i] = df1[' 订货包数'][i] + 1
C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\2634855962.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['订货\n类型'][i] = 3
C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\2634855962.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  if 0 < ((df1['门店\n库存'][i]+df1[' 订货包数'][i]*df1['规格'][i])/df1['本店\n日均销'][i]) < standard_number_of_day_down-advance_day and df1['本店\n日均销'][i] > sell_well_down and (df1['门店\n库存'][i]+(df1[' 订货包数'][i]+1)*df1['规格'][i])

In [87]:
sale_number = []  # 下单包数
for i in range(df1.shape[0]):
    if df1['陈列类'][i] == '购物袋':
        sale_number.append(0)
    else:
        sale_number.append(df1[' 订货包数'][i])
        
df1[' 订货包数'] =sale_number

In [88]:
# 多退少补

    # 在范围里面
if sum(df1[' 订货包数']*df1['规格']) >stock_piece-patch_Upper_and_lower and sum(df1[' 订货包数']*df1['规格']) <stock_piece+patch_Upper_and_lower:
    print('##################  检  查  一  下  ##################')

    
    
    
    # 如果大了
elif sum(df1[' 订货包数']*df1['规格']) >stock_piece+patch_Upper_and_lower:
    
    print('##################  大  了  ##################')
    
    Sale_Day = []  #改可售天数，提高效率
    
    for i in range(df1.shape[0]):
        print('更新可售天数',i)
        if df1['订货\n类型'][i] ==0 or df1['订货\n类型'][i] ==3:
            Sale_Day.append((df1['门店\n库存'][i]+df1[' 订货包数'][i]*df1['规格'][i])/max(df1['本店\n日均销'][i],df1['综合\n日均销'][i]))
        elif df1['订货\n类型'][i] ==1:
            Sale_Day.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['综合\n日均销'][i])
        elif df1['订货\n类型'][i] ==2:
            Sale_Day.append((df1['门店\n库存'][i]+sale_number_1*df1['规格'][i])/df1['区域\n日均销'][i])
        else:
            Sale_Day.append(0)
            
    df1['可售天数'] = Sale_Day     
    # 订货总额
    
    # money_amount = sum(df1[' 订货包数']*df1['规格']*df1['本地\n组合价'])
    
    sale_type_1 = [2,3]
    sale_type_2 = ['区域日均销','本店最后补的']
    sale_type_4 = ['区域\n日均销','本店\n日均销']
    
    for i in range(len(sale_type_1)):
        
        # 判断用
        sale_type_3 = 0 
        True_False = True
        
        for k in range(df1.shape[0]):
            print(sale_type_2[i],'算补货了的SKU',k)
            if df1['订货\n类型'][k] == sale_type_1[i] and df1[' 订货包数'][k] > 0:
                sale_type_3 = sale_type_3 + df1[' 订货包数'][k]   
        while sale_type_3 > 0 and True_False :
            df1=df1.sort_values(by=['可售天数'],ascending=False)
            #df1=df1.reset_index(drop = True)
            for j in range(df1.shape[0]):
                print(sale_type_2[i],'去天数',j,sum(df1[' 订货包数']*df1['规格']),sale_type_3)
                if df1['订货\n类型'][j] == sale_type_1[i] and df1[' 订货包数'][j] > 0:
                    df1[' 订货包数'][j] = df1[' 订货包数'][j] - 1
                    sale_type_3 = sale_type_3 - 1
                    df1['可售天数'][j] = (df1['门店\n库存'][j]+sale_number_1*df1['规格'][j])/df1[sale_type_4[i]][j]
                if sum(df1[' 订货包数']*df1['规格']) <stock_piece+patch_Upper_and_lower:
                    True_False = False
                    break
        if sum(df1[' 订货包数']*df1['规格']) <stock_piece+patch_Upper_and_lower:
            break
    
    if sum(df1[' 订货包数']*df1['规格']) >stock_piece+patch_Upper_and_lower:
        wh = 0
        while sum(df1[' 订货包数']*df1['规格']) > stock_piece+patch_Upper_and_lower and wh!=df1.shape[0]-1:
            print('最后降序',sum(df1[' 订货包数']*df1['规格']),wh)
            
            df1=df1.sort_values(by=['可售天数'],ascending=False)
            #df1=df1.reset_index(drop = True)
            wheel = 0
            for wh in range(df1.shape[0]):
                if max(df1['本店\n日均销'][wh],df1['综合\n日均销'][wh]) < sell_well_down:
                    if df1[' 订货包数'][wh] > 0 and df1['订货\n类型'][wh] != '新品':
                        df1[' 订货包数'][wh] = df1[' 订货包数'][wh] - 1
                        df1['可售天数'][wh] = (df1['门店\n库存'][wh]+df1[' 订货包数'][wh]*df1['规格'][wh])/max(df1['本店\n日均销'][wh],df1['综合\n日均销'][wh])                  
                        wheel = wheel + 1 
                    if wheel == wheel_number:
                        break
                else:
                    if df1[' 订货包数'][wh] > 0 and df1['订货\n类型'][wh] != '新品' and (df1['门店\n库存'][wh]+(df1[' 订货包数'][wh]-1)*df1['规格'][wh])/max(df1['本店\n日均销'][wh],df1['综合\n日均销'][wh])>standard_number_of_day_down:
                        df1[' 订货包数'][wh] = df1[' 订货包数'][wh] - 1
                        df1['可售天数'][wh] = (df1['门店\n库存'][wh]+df1[' 订货包数'][wh]*df1['规格'][wh])/max(df1['本店\n日均销'][wh],df1['综合\n日均销'][wh])                  
                        wheel = wheel + 1 
                    if wheel == wheel_number:
                        break

                
                    
    print('##################  完 成 多 降 少  ##################')

    
    
    
    # 如果小了
elif sum(df1[' 订货包数']*df1['规格']) <stock_piece-patch_Upper_and_lower:
    print('##################  小  了  ##################')
    # 加新品
    for i in range(df1.shape[0]):
        if df1['仓库\n可用库存'][i] > 0 and df1['订货\n类型'][i]==' 新品' and df1['订货\n原因'][i] != '禁配' and df1[' 订货包数'][i] == 0 and df1['细类\n平衡率'] < mini_balance and df1['细类偏差\nPCS'] < slender_deviate:
            df1[' 订货包数'] = 1
            df1['订货\n类型'][i] = -2
    print('加新品：'+str(sum(df1[' 订货包数']*df1['规格'])))


    
    
    # 加系统补货
    if sum(df1[' 订货包数']*df1['规格']) <stock_piece:
        df1=df1.sort_values(by=['综合\n日均销'],ascending=True)
        #df1=df1.reset_index(drop = True)
        for i in range(df1.shape[0]):
            if df1['仓库\n可用库存'][i] > 0 and df1['订货\n类型'][i]!='新品' and df1['订货\n原因'][i] != '禁配':
                while df1['系统\n订货数量'][i] > df1[' 订货包数'][i]*df1['规格'][i] and (df1['门店\n库存'][i]+(df1[' 订货包数'][i]+1)*df1['规格'][i])/df1['综合\n日均销'][i] < max_sale_day:
                    df1[' 订货包数'][i] = df1[' 订货包数'][i] + 1 
                    df1['订货\n类型'][i] = 4
            if df1['仓库\n可用库存'][i] > 0 and df1['订货\n类型'][i]!='新品' and df1['门店\n库存'][i] == 0 and df1['系统\n订货数量'][i] > 0 and df1[' 订货包数'][i] == 0 and df1['区域\n日均销'][i] > add_to_down and (df1['门店\n库存'][i]+1*df1['规格'][i])/df1['区域\n日均销'][i] < max_sale_day:
                df1[' 订货包数'][i] = 1 
                df1['订货\n类型'][i] = 5
            if sum(df1[' 订货包数']*df1['规格']) > stock_piece: 
                break 
                
        print('加系统补货: '+str(sum(df1[' 订货包数']*df1['规格']*df1['本地\n组合价'])))
        
    
    
    # 加畅销  
    if sum(df1[' 订货包数']*df1['规格']) <stock_piece:
        # 订货总额
        # money_amount = sum(df1[' 订货包数']*df1['规格']*df1['本地\n组合价'])
        
        df1=df1.sort_values(by=['综合\n日均销'],ascending=False)
        #df1=df1.reset_index(drop = True)
        
        True_False = True
        while sum(df1[' 订货包数']*df1['规格']) <stock_piece and standard_number_of_day_down < big_sell_well_day and True_False:
            standard_number_of_day_down = standard_number_of_day_down + 1
            standard_number_of_day_up = standard_number_of_day_up + 1
                        
            for i in range(df1.shape[0]):
                if sum(df1[' 订货包数']*df1['规格']) <stock_piece:
                    if df1['仓库\n可用库存'][i] > 0 and df1['订货\n类型'][i]!='新品' and df1['订货\n原因'][i] != '禁配' and df1['综合\n日均销'][i] > add_to_down: #and df1['细类\n平衡率'] < mini_balance :
                        while (df1['门店\n库存'][i]+(df1[' 订货包数'][i])*df1['规格'][i])/df1['综合\n日均销'][i] < standard_number_of_day_down :
                            if df1['仓库\n可用库存'][i] >= (df1[' 订货包数'][i] + 1)*df1['规格'][i] and (df1['门店\n库存'][i]+(df1[' 订货包数'][i]+1)*df1['规格'][i])/df1['综合\n日均销'][i] < standard_number_of_day_up:
                                df1[' 订货包数'][i] = df1[' 订货包数'][i] + 1
                                df1['订货\n类型'][i] = 6
                                # money_amount = money_amount + 1*df1['规格'][i]*df1['本地\n组合价'][i]
                            else:
                                break
                                
                else:
                    break
                    
            print('畅销最低周转天数为： '+str(standard_number_of_day_down)+'天  '+ str(sum(df1[' 订货包数']*df1['规格'])))
            
            if sum(df1[' 订货包数']*df1['规格']) > stock_piece: 
                break    
    
    
            
                
        

                
    # 加入细类有偏差的
    # if sum(df1[' 订货包数']*df1['规格']*df1['本地\n组合价']) <stock_amount-patch_Upper_and_lower:
    #     df1=df1.sort_values(by=['细类\n平衡率'],ascending=False)
    #     for i in range(df1.shape[0]):
    #         if df1['细类\n平衡率'][i] > 0 and df1['细类\n平衡率'][i] < 1 and 品类是否补够了:
    #             加入细类的整体情况订货情况，判断哪些品类还要补
    #             
    print('##################  完 成 少 补 多  ##################')
    
print('正常最后补充数量：',sum(df1[' 订货包数']*df1['规格']))

##################  小  了  ##################
加新品：723


C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\786197571.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[' 订货包数'][i] = df1[' 订货包数'][i] + 1
C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\786197571.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['订货\n类型'][i] = 4
C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\786197571.py:112: RuntimeWarning: divide by zero encountered in double_scalars
  while df1['系统\n订货数量'][i] > df1[' 订货包数'][i]*df1['规格'][i] and (df1['门店\n库存'][i]+(df1[' 订货包数'][i]+1)*df1['规格'][i])/df1['综合\n日均销'][i] < max_sale_day:


加系统补货: 19565.340000000007


C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\786197571.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[' 订货包数'][i] = df1[' 订货包数'][i] + 1
C:\Users\Irene\AppData\Local\Temp\ipykernel_16060\786197571.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['订货\n类型'][i] = 6


畅销最低周转天数为： 41天  3605
畅销最低周转天数为： 42天  3665
畅销最低周转天数为： 43天  3712
畅销最低周转天数为： 44天  3781
畅销最低周转天数为： 45天  3809
畅销最低周转天数为： 46天  3871
畅销最低周转天数为： 47天  3982
畅销最低周转天数为： 48天  4017
畅销最低周转天数为： 49天  4097
畅销最低周转天数为： 50天  4152
畅销最低周转天数为： 51天  4223
畅销最低周转天数为： 52天  4291
畅销最低周转天数为： 53天  4384
畅销最低周转天数为： 54天  4469
畅销最低周转天数为： 55天  4568
畅销最低周转天数为： 56天  4660
畅销最低周转天数为： 57天  4787
畅销最低周转天数为： 58天  5082
畅销最低周转天数为： 59天  5179
畅销最低周转天数为： 60天  5250
畅销最低周转天数为： 61天  5368
畅销最低周转天数为： 62天  5527
##################  完 成 少 补 多  ##################
正常最后补充数量： 5527


In [89]:
# 判断购物袋要不要补

# 更新索引
#df1 = df1.reset_index(drop = True)

#for i in range(df1.shape[0]):
#    while df1['陈列类'][i] == '装材购物袋' and df1['订货\n原因'][i] != '禁配' and (df1['门店\n库存'][i]+(df1[' 订货包数'][i])*df1['规格'][i])/df1['综合\n日均销'][i] < standard_number_of_day_down and df1['仓库\n可用库存'][i] >= (df1[' 订货包数'][i] + 1)*df1['规格'][i] and (df1['门店\n库存'][i]+(df1[' 订货包数'][i]+1)*df1['规格'][i])/df1['综合\n日均销'][i] < standard_number_of_day_up:
#                df1[' 订货包数'][i] = df1[' 订货包数'][i] + 1
#                df1['订货\n类型'][i] = 6

In [90]:
print(sum(df1[' 订货包数']*df1['规格']))
print(store_code)

5527
US23


In [91]:
df1['调整后\n订货数量']= df1[' 订货包数']*df1['规格']

In [92]:
# 导出文件
df2=df1.sort_index(ascending=True)
df2.to_excel(store_code+' '+date+' '+'下单数量.xlsx',index=False)

print('##################  完   成  ##################')
#elapsed = (time.clock() - start)
#print("Time used:",elapsed)

##################  完   成  ##################


In [93]:
# 剔除有订货包数但是销量很差的
# 小于15天根据品类平衡如何判断
